In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 灰度转换
def convert_to_grayscale(image):
    print("灰度转换已完成 10%")
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 滤波
def apply_gaussian_blur(image, kernel_size=5):
    print("高斯滤波已完成 20%")
    return cv2.GaussianBlur(image, (kernel_size, kernel_size), 0)

# 边缘检测
def detect_edges(image, low_threshold=50, high_threshold=150):
    print("边缘检测已完成 30%")
    return cv2.Canny(image, low_threshold, high_threshold)

# 提取感兴趣区域ROI：一开始尝试定义为图像下半部分的一个三角形区域。
# 改进后，ROI被定义成一个更符合实际车道形状的梯形区域。
def region_of_interest(image):
    height = image.shape[0]
    width = image.shape[1]
    polygons = np.array([
        [
            (int(width * 0.1), height),
            (int(width * 0.9), height),
            (int(width * 0.55), int(height * 0.6)),
            (int(width * 0.45), int(height * 0.6))
        ]
    ])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    print("提取感兴趣区域已完成 40%")
    return masked_image

# 霍夫变换检测直线：通过降低 minLineLength （从100降到20）和增加 maxLineGap （从50增加到300）等，
# 霍夫变换能够检测到更短的线段并允许更大的断裂，从而提高了检测灵敏度。
def hough_transform(image):
    lines = cv2.HoughLinesP(image, 1, np.pi / 180, 20, minLineLength=20, maxLineGap=300)
    print("霍夫变换检测直线已完成 60%")
    return lines

# 线段后处理：对检测到的线段进行后处理，计算左右车道线的平均斜率和截距；
# 并对检测到的线段进行合并，减少了由于噪声和断裂导致的检测不连续性，使检测结果更加稳定和连续。
def average_slope_intercept(image, lines):
    left_lines = []  # (slope, intercept)
    right_lines = []  # (slope, intercept)
    left_weights = []  # length of the line segment
    right_weights = []  # length of the line segment

    for line in lines:
        for x1, y1, x2, y2 in line:
            if x1 == x2:
                continue  # skip vertical lines
            slope = (y2 - y1) / (x2 - x1)
            intercept = y1 - slope * x1
            length = np.sqrt((y2 - y1) ** 2 + (x2 - x1) ** 2)
            if slope < 0:  # left lane
                left_lines.append((slope, intercept))
                left_weights.append(length)
            else:  # right lane
                right_lines.append((slope, intercept))
                right_weights.append(length)

    left_lane = np.dot(left_weights, left_lines) / np.sum(left_weights) if len(left_weights) > 0 else None
    right_lane = np.dot(right_weights, right_lines) / np.sum(right_weights) if len(right_weights) > 0 else None
    print("线段后处理已完成 80%")
    return left_lane, right_lane

def make_line_points(y1, y2, line):
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept) / slope)
    x2 = int((y2 - intercept) / slope)
    y1 = int(y1)
    y2 = int(y2)
    return ((x1, y1), (x2, y2))

def draw_lines(image, lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        left_lane, right_lane = average_slope_intercept(image, lines)
        y1 = image.shape[0]
        y2 = y1 * 0.6
        left_line = make_line_points(y1, y2, left_lane)
        right_line = make_line_points(y1, y2, right_lane)
        for line in [left_line, right_line]:
            if line is not None:
                cv2.line(line_image, *line, (0, 255, 0), 10)  # 绿色线条
    print("车道线绘制已完成 90%")
    return line_image

# 处理单帧图像
def process_image(image):
    gray = convert_to_grayscale(image)
    blurred = apply_gaussian_blur(gray)
    edges = detect_edges(blurred)
    roi = region_of_interest(edges)
    lines = hough_transform(roi)
    line_image = draw_lines(image, lines)
    combined = cv2.addWeighted(image, 0.8, line_image, 1, 1)
    print("单帧图像处理已完成 100%")
    return combined

# 处理视频
def process_video(input_video_path, output_video_path):
    cap = cv2.VideoCapture(input_video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_image(frame)
        out.write(processed_frame)
        print("视频处理进度: {:.2f}%".format((cap.get(cv2.CAP_PROP_POS_FRAMES) / cap.get(cv2.CAP_PROP_FRAME_COUNT)) * 100))

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print("视频处理已完成 100%")

# 处理视频
process_video('drive.mp4', 'output_drive_V3.mp4')

灰度转换已完成 10%
高斯滤波已完成 20%
边缘检测已完成 30%
提取感兴趣区域已完成 40%
霍夫变换检测直线已完成 60%
线段后处理已完成 80%
车道线绘制已完成 90%
单帧图像处理已完成 100%
视频处理进度: 0.47%
灰度转换已完成 10%
高斯滤波已完成 20%
边缘检测已完成 30%
提取感兴趣区域已完成 40%
霍夫变换检测直线已完成 60%
线段后处理已完成 80%
车道线绘制已完成 90%
单帧图像处理已完成 100%
视频处理进度: 0.93%
灰度转换已完成 10%
高斯滤波已完成 20%
边缘检测已完成 30%
提取感兴趣区域已完成 40%
霍夫变换检测直线已完成 60%
线段后处理已完成 80%
车道线绘制已完成 90%
单帧图像处理已完成 100%
视频处理进度: 1.40%
灰度转换已完成 10%
高斯滤波已完成 20%
边缘检测已完成 30%
提取感兴趣区域已完成 40%
霍夫变换检测直线已完成 60%
线段后处理已完成 80%
车道线绘制已完成 90%
单帧图像处理已完成 100%
视频处理进度: 1.87%
灰度转换已完成 10%
高斯滤波已完成 20%
边缘检测已完成 30%
提取感兴趣区域已完成 40%
霍夫变换检测直线已完成 60%
线段后处理已完成 80%
车道线绘制已完成 90%
单帧图像处理已完成 100%
视频处理进度: 2.34%
灰度转换已完成 10%
高斯滤波已完成 20%
边缘检测已完成 30%
提取感兴趣区域已完成 40%
霍夫变换检测直线已完成 60%
线段后处理已完成 80%
车道线绘制已完成 90%
单帧图像处理已完成 100%
视频处理进度: 2.80%
灰度转换已完成 10%
高斯滤波已完成 20%
边缘检测已完成 30%
提取感兴趣区域已完成 40%
霍夫变换检测直线已完成 60%
线段后处理已完成 80%
车道线绘制已完成 90%
单帧图像处理已完成 100%
视频处理进度: 3.27%
灰度转换已完成 10%
高斯滤波已完成 20%
边缘检测已完成 30%
提取感兴趣区域已完成 40%
霍夫变换检测直线已完成 60%
线段后处理已完成 80%
车道线绘制已完成 90%
单帧图像处理已完成 100%
视频处理进度: 3.74%
灰度转换已完成 10%
高斯滤波已完成 20%
